In [13]:
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [16]:
EPOCHS = 200
LR = 1e-2
BS = 16384
SEED = 41

In [17]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED) # Seed 고정

In [18]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [31]:
class AutoEncoder(nn.Module):
    def __init__(self, dim):
        super(AutoEncoder, self).__init__()
        self.Encoder = nn.Sequential(
            nn.Linear(dim,dim*4),
            nn.BatchNorm1d(dim*4),
            nn.LeakyReLU(),
            nn.Linear(dim*4, dim*8),
            nn.BatchNorm1d(dim*8),
            nn.LeakyReLU(),
        )
        self.Decoder = nn.Sequential(
            nn.Linear(dim*8,dim*4),
            nn.BatchNorm1d(dim*4),
            nn.LeakyReLU(),
            nn.Linear(dim*4,dim),
        )
        
    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x


In [19]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        # Loss Function
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self, ):
        self.model.to(self.device)
        best_score = 0
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                x = x.float().to(self.device)
                self.optimizer.zero_grad()

                _x = self.model(x)
                loss = self.criterion(x, _x)

                loss.backward()
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 0.95)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score < score:
                best_score = score
                torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
    
    def validation(self, eval_model, thr):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)

                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

In [20]:
def prediction(model, thr, test_loader, device):
    model.to(device)
    model.eval()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pred = []
    with torch.no_grad():
        for x in iter(test_loader):
            x = x.float().to(device)
            
            _x = model(x)
            
            diff = cos(x, _x).cpu().tolist()
            batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
            pred += batch_pred
    return pred

In [25]:
train_df = pd.read_csv('C:/Users/김승우/Desktop/파이썬/train.csv')
train_df = train_df.drop(columns=['ID',"V13","V15","V19","V20","V22","V23","V24","V25","V26","V28","V29","V30"])
val_df = pd.read_csv('C:/Users/김승우/Desktop/파이썬/val.csv')
val_df = val_df.drop(columns=['ID',"V13","V15","V19","V20","V22","V23","V24","V25","V26","V28","V29","V30"])

DIM = train_df.shape[1]

In [32]:
val_df.iloc[:,[0,2]]

,V1,V3
0,-0.338262,1.044367
1,0.962496,-0.171479
2,1.145524,0.194008
3,0.927060,0.387585
4,-3.005237,1.483691
...,...,...
28457,-0.546378,-0.313252
28458,-0.764523,-0.907599
28459,-0.446951,-0.168583
28460,2.039560,-1.196825


In [46]:
f_index=list(range(30))
f_index.remove(17)
print(f_index)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


In [ ]:
for i in f_index :
    

In [26]:
train_dataset = MyDataset(df=train_df, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=0)

val_dataset = MyDataset(df = val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=0)

In [30]:
model = nn.DataParallel(AutoEncoder(DIM))
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

Epoch : [0] Train loss : [0.509979614189693] Val Score : [0.09039792798036007])
Epoch : [1] Train loss : [0.26519714082990375] Val Score : [0.4423818697557707])
Epoch : [2] Train loss : [0.18218669721058436] Val Score : [0.49759503782856607])
Epoch : [3] Train loss : [0.14219713423933303] Val Score : [0.5198745135121873])
Epoch : [4] Train loss : [0.12081823178700038] Val Score : [0.5426217169100375])
Epoch : [5] Train loss : [0.10976027590887887] Val Score : [0.6249581098151419])
Epoch : [6] Train loss : [0.09959210987601962] Val Score : [0.660365522073326])
Epoch : [7] Train loss : [0.09298927336931229] Val Score : [0.7226686263465465])
Epoch : [8] Train loss : [0.08883456460067204] Val Score : [0.7684388896488608])
Epoch : [9] Train loss : [0.08633469045162201] Val Score : [0.8009145358549308])
Epoch : [10] Train loss : [0.08445163922650474] Val Score : [0.8674887641844412])
Epoch : [11] Train loss : [0.07980200116123472] Val Score : [0.890501890608512])
Epoch : [12] Train loss : [0

KeyboardInterrupt: 

In [11]:
model = AutoEncoder(DIM)
model.load_state_dict(torch.load('./best_model.pth'))
model = nn.DataParallel(model)
model.eval()

DataParallel(
  (module): AutoEncoder(
    (Encoder): Sequential(
      (0): Linear(in_features=18, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=64, out_features=128, bias=True)
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
    )
    (Decoder): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=64, out_features=18, bias=True)
    )
  )
)

18

In [ ]:
test_df = pd.read_csv('C:/Users/김승우/Desktop/파이썬/test.csv')
test_df = test_df.drop(columns=['ID'])

In [ ]:
test_dataset = MyDataset(test_df, False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=0)

In [ ]:
preds = prediction(model, 0.95, test_loader, device)

In [ ]:
submit = pd.read_csv('C:/Users/김승우/Desktop/파이썬/sample_submission.csv')
submit['Class'] = preds
submit.to_csv('./submit_autoencoder.csv', index=False)